# Data processing

# 1. Digital PDF

## 1.1. Loading the PDF

In [1]:
import pdfplumber

print(pdfplumber.__version__)

0.6.2


In [ ]:
import os

proj_name = "electrificat"
curr_dir = os.getcwd()
if proj_name in curr_dir:
    new_dir = curr_dir.split(proj_name)[0]
    new_dir = os.path.join(new_dir, proj_name)
    os.chdir(new_dir)

os.getcwd()

In [ ]:
root = os.getcwd()
pdf_file = os.path.join(root, "data/raw/Factura_FE22321385375221.pdf")
# pdf_file = os.path.join(root, "data/raw/HOLALUZ_628bdcf527c3c2.13969730.pdf")

In [ ]:
pdf = pdfplumber.open(pdf_file)

In [ ]:
# len(pdf.pages)
p1 = pdf.pages[1]

In [ ]:
im = p1.to_image()
im

## 1.2. Extract the text

In [ ]:
extracted_text = [page.extract_text() for page in pdf.pages]
extracted_text = "\n".join(extracted_text)
# extracted_text = pdf.pages[1].extract_text()

### 1.2.1 Extract maximum consumed power

In [ ]:
def extract_target(text: str, target: list) -> list:
    """
    Return a vector of the explicit target words found on the provided text.
    :param text:
    :param target:
    :return:
    """
    found_checker = [False] * len(target)
    found_words = {}

    for i, list_word in enumerate(target):
        words = []
        for word in list_word:
            if word in text:
                found_checker[i] = True
                print(word)
                words.append(word)
                found_words[i] = words

    if sum(found_checker) == len(target):
        print("Target words found!")
        return found_words

    # return [word for list_word in target for word in list_word if word in text]
    print("Keywords not found.")
    return []

In [ ]:
def extract_line(text: str, target: list) -> str:
    """
    Extract the line where the most of the target words are located
    :param text:
    :param target:
    :return:
    """
    # assert target[0] in text, "The target words are not present in the provided text"

    # Parse line splits to make a list of str (one line per slot)
    extracted_lines = text.splitlines()
    # found_target = [i for i,curr_line in enumerate(extracted_lines) for word in target if word in curr_line]
    # print(found_target)

    for curr_line in extracted_lines:
        # print(curr_line)
        if all(target[x] in curr_line for x in target):
            return curr_line
    # common = Counter(found_target).most_common()

    # for i,count in enumerate(common):
    #     print(i,count)
    #     print(len(target))
    #     if count[1] <= len(target):
    #         line_nr = count[0]
    #         print(line_nr)
    #         break

    # line_nr = Counter(found_target).most_common()[0][0]

    return extracted_lines[line_nr]

In [ ]:
# for x in found:
#     print(found[x])
#
# for curr_line in extracted_lines:
#
#     if all(target[x] in curr_line for x in target):
#     return curr_line
extracted_lines = extracted_text.splitlines()
#
from itertools import permutations

# permutations()
# for curr_line in extracted_lines:
#     if all(found[key] in curr_line for key in found)
#     # print(a)
vals = {}
myval = []
lengths = [len(val) for val in found.values()]

permutes = 1
for x in lengths:
    permutes = permutes * x
print(permutes)

for key, value in found.items():
    print(f"key: {key} , value: {value}")


for key in found:
    myval.append(found[key][0])

vals[0] = myval
myval = []

for val in lengths:
    if val > 1:
        max_dict = len(vals.keys())  # this is +1, starts at 0

        for key, value in found.items():
            if len(value) > 1:
                myval.append(found[key][max_dict])
            else:
                myval.append(found[key][0])

vals[max_dict] = myval
myval = []

print(vals)

In [ ]:
from itertools import permutations, product

result = []
for perm in permutations([found[0], found[1]]):
    for prod in product(*perm):
        if prod[0] in found[0]:
            result.append(prod)
print(result)

In [ ]:
# List of words of interest in order to find the desired section
power = {
    "potencia",
    "potència",
    "poténcia",
    "Potencia",
    "Potència",
    "Poténcia",
    "potencies",
    "potencias",
    "potències",
    "poténcias",
    "Potencias",
    "Potències",
    "Poténcias",
}
maxima = {
    "maxima",
    "màxima",
    "máxima",
    "Maxima",
    "Màxima",
    "Máxima",
    "maximes",
    "maximas",
    "màximes",
    "máximas",
    "Maximas",
    "Màximes",
    "Máximas",
}
year = ["any", "año", "Any", "Año"]
words = [power, maxima, year]

In [ ]:
found = extract_target(extracted_text, words)
found

In [ ]:
target_line = extract_line(extracted_text, found)
target_line

In [ ]:
def extract_target_lines(line: str) -> list:
    """
    Return a stripped line with the desired P1, P2, P3 power count
    :param line:
    :return:
    """

    p1 = p2 = p3 = None

    if "P1" in line:
        p1 = line.split("P1")[0]
        line = line.split("P1")[-1]  # Not change the order

    if "P2" in line:
        p2 = line.split("P2")[0]

    if "P3" in line:
        line = line.split("P2")[-1]
        p3 = line.split("P3")[0]

    return [p1, p2, p3]

In [ ]:
def extract_numbers(text: str) -> (float, None):
    """
    Input some text and extract a single number as a float
    :param text:
    :return:
    """
    import re

    assert (
        len(re.findall(r"\d+", text)) > 0
    ), "The input string does not contain any number"

    # Replace , with . for decimal position, if found.
    if "," in text:
        text = text.replace(",", ".")

    if "." in text:
        number = re.findall(r"\d+\.\d+", text)
        if float(number[0]) == 0.0:
            return None

        return float(number[0])

    # When there's no decimal position
    number = re.findall(r"\d+", text)

    if number[0] == 0:
        return None

    return number[0]

In [ ]:
def extract_pmax(p_range: list) -> list:
    """
    From the lines identified with P1,P2,P3, extract the numerical value.
    Example: "Here you have P1 0.34 kWh and P2 for 10,00 kwh" -> [0.34, 10.0]
    :param p_range:
    :return:
    """
    pmax = [
        extract_numbers(px)
        for px in p_range
        if px is not None and extract_numbers(px) is not None
    ]

    [print(f"P{i+1} maximum power: {pmax[i]} kWh") for i in range(len(pmax))]
    return pmax

In [ ]:
p_ranges = extract_target_lines(target_line)
print(p_ranges)
p_max = extract_pmax(p_ranges)

### 1.2.2. Extract total consumed power

In [ ]:
total = {"total", "Total", "TOTAL"}
consumed = {
    "consum",
    "Consum",
    "CONSUM",
    "consumo",
    "Consumo",
    "CONSUMO",
    "consumit",
    "Consumit",
    "CONSUMIT",
    "consumido",
    "Consumido",
    "CONSUMIDO",
    "consumida",
    "Consumida",
    "CONSUMIDA",
}

words = [total, consumed]

In [ ]:
found = extract_target(extracted_text, words)
print(found)
target_line = extract_line(extracted_text, found)

In [ ]:
target_line

In [ ]:
total_consumed = extract_numbers(target_line)
print(total_consumed)
print(f"Total energy consumed: {total_consumed} kWh")

### 1.2.3. Extract total price paid

In [ ]:
total = {"total", "Total", "TOTAL"}
pagar = {"pagar", "Pagar", "PAGAR"}
words = [total, pagar]

In [ ]:
found = extract_target(extracted_text, words)
target_line = extract_line(extracted_text, found)

In [52]:
total_paid = extract_numbers(target_line)
print(f"Total paid for the bill: {total_paid}€")

Total paid for the bill: 1€


### 1.2.4. Extract prices paid for different power ranges